<a href="https://colab.research.google.com/github/JALVARADORUIZ/UB_Proyecto/blob/main/TFM_Pre_procesado_de_Im%C3%A1genes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### **PREPARACIÓN DEL ENTORNO**


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
from keras.preprocessing.image import load_img 
from keras.preprocessing.image import img_to_array 
from keras.applications.vgg16 import preprocess_input 

In [ ]:
from keras.applications.vgg16 import VGG16 
from keras.models import Model

In [ ]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
import pickle

#### **PREPROCESADO DE IMÁGENES**


In [ ]:
path = r"/content/drive/MyDrive/TFM_Máster BDDS/datawine/vintage-images"
os.chdir(path)


In [ ]:
wines = []

with os.scandir(path) as files:
    for file in files:
        wines.append(file.name)

In [ ]:

img = load_img(wines[0], target_size=(224,224))

img = np.array(img)

print(img.shape)

(224, 224, 3)


/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [ ]:
reshaped_img = img.reshape(1,224,224,3)
print(reshaped_img.shape)


(1, 224, 224, 3)


In [ ]:
x = preprocess_input(reshaped_img)

In [ ]:
model = VGG16()
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

In [ ]:
def extract_features(file, model):
    # load the image as a 224x224 array
    img = load_img(file, target_size=(224,224))
    # convert from 'PIL.Image.Image' to numpy array
    img = np.array(img) 
    # reshape the data for the model reshape(num_of_samples, dim 1, dim 2, channels)
    reshaped_img = img.reshape(1,224,224,3) 
    # prepare image for model
    imgx = preprocess_input(reshaped_img)
    # get the feature vector
    features = model.predict(imgx, use_multiprocessing=True)
    return features

In [ ]:
data = {}
p = r"/content/drive/MyDrive/TFM_Máster BDDS/datawine/wine_features.pkl"

# loop through each image in the dataset
for wine in wines:
    # try to extract the features and update the dictionary
    try:
        feat = extract_features(wine,model)
        data[wine] = feat
    # if something fails, save the extracted features as a pickle file (optional)
    except:
        with open(p,'wb') as file:
            pickle.dump(data,file)
          
 
# get a list of the filenames
filenames = np.array(list(data.keys()))

In [ ]:
# get a list of just the features
feat = np.array(list(data.values()))
feat.shape

(7618, 1, 4096)

In [ ]:
# reshape so that there are 210 samples of 4096 vectors
feat = feat.reshape(-1,4096)
feat.shape

(7618, 4096)

In [ ]:
df_labels = pd.read_csv('/content/drive/MyDrive/TFM_Máster BDDS/datawine/df_clean_europe_nooutliers.csv')

In [ ]:
df_labels.shape

(5462, 15)

In [ ]:
df_images = pd.DataFrame(feat)

In [ ]:
df_filenames = pd.DataFrame(filenames, columns=['image_clean'])


In [ ]:
df_imagesf = pd.concat([df_filenames,df_images], axis=1)

In [ ]:
dff = df_labels.merge(df_imagesf, how='left', on ='image_clean')

In [ ]:
dff.head(1)

,winery,product,region,country_code,rating,variety,price_usd,image,continente,country_code_clean,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,louis jadot,louis jadot corton grand cru greves 2006,corton grand cru,fr,4.0,pinot noir,89.99,vintage-images/TJyGXkvdQsSBDVqT8hHDtg_pl_480x6...,europe,france,...,0.0,0.0,0.114859,1.35443,0.0,0.0,0.158488,0.0,0.696759,2.084952


In [ ]:
len(dff)

5462

In [ ]:
dff[dff.isna().any(axis=1)]

,winery,product,region,country_code,rating,variety,price_usd,image,continente,country_code_clean,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
258,cavit,cavit lunetta prosecco 2008,prosecco,it,4.0,glera,13.990000,vintage-images/w5Jv81C1TOCF1SiHc0WyRQ_pl_480x6...,europe,italy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1509,chateau l enclos,chateau l enclos pomerol 1998,pomerol,fr,4.5,cabernet franc,79.900000,vintage-images/20LQdeDWS5qhzvoFE7A83w_pl_480x6...,europe,france,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2243,cavit,cavit lunetta prosecco,prosecco,it,4.0,glera,11.990000,vintage-images/w5Jv81C1TOCF1SiHc0WyRQ_pl_480x6...,europe,italy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2851,bastianich,bastianich sauvignon b 2010,colli orientali del friuli,it,5.0,sauvignon blanc,19.990000,vintage-images/000250llnn0ft.jpg,europe,italy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3746,chateau l enclos,chateau l enclos pomerol,pomerol,fr,4.5,cabernet franc,78.974881,vintage-images/20LQdeDWS5qhzvoFE7A83w_pl_480x6...,europe,france,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
dff1 = dff.drop([258,1509,2243,2851,3746],axis=0)

In [ ]:
dff1[dff1.isna().any(axis=1)]

,winery,product,region,country_code,rating,variety,price_usd,image,continente,country_code_clean,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095


In [ ]:
dff1.to_csv('/content/drive/MyDrive/TFM_Máster BDDS/datawine/df_img_processed.csv', index=False)